## Week1 Work
### Importing necessary libraries

In [72]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Importing the data from txt files into pandas dfs

In [73]:
files_to_import = ['../aapl.us.txt', '../msft.us.txt', '../amzn.us.txt', '../tsla.us.txt', '../googl.us.txt']
dataframes = {file: pd.read_csv(file) for file in files_to_import}

aapl_df = dataframes['../aapl.us.txt']
msft_df = dataframes['../msft.us.txt']
amzn_df = dataframes['../amzn.us.txt']
tsla_df = dataframes['../tsla.us.txt']
googl_df = dataframes['../googl.us.txt']

In [74]:
aapl_df['Ticker'] = 'AAPL'
msft_df['Ticker'] = 'MSFT'
amzn_df['Ticker'] = 'AMZN'
tsla_df['Ticker'] = 'TSLA'
googl_df['Ticker'] = 'GOOGL'

### Converting the date column to datetime variable and setting the df to be multi-indexed based on Ticker and Date

In [75]:
for df in [aapl_df, msft_df, amzn_df, tsla_df, googl_df]:
    df['Date'] = pd.to_datetime(df['Date'])
for df in [aapl_df, msft_df, amzn_df, tsla_df, googl_df]:
    df.set_index(['Ticker', 'Date'], inplace=True)

### Dropping openint column and combining dfs

In [76]:
df_multi = pd.concat([aapl_df, msft_df, amzn_df, tsla_df, googl_df])
df_multi.sort_index(inplace=True)
df_multi = df_multi.drop(columns='OpenInt')
df_multi

Open       High        Low      Close    Volume
Ticker Date                                                            
AAPL   1984-09-07    0.42388    0.42902    0.41874    0.42388  23220030
       1984-09-10    0.42388    0.42516    0.41366    0.42134  18022532
       1984-09-11    0.42516    0.43668    0.42516    0.42902  42498199
       1984-09-12    0.42902    0.43157    0.41618    0.41618  37125801
       1984-09-13    0.43927    0.44052    0.43927    0.43927  57822062
...                      ...        ...        ...        ...       ...
TSLA   2017-11-06  307.00000  307.50000  299.01000  302.78000   6482486
       2017-11-07  301.02000  306.50000  300.03000  306.05000   5286320
       2017-11-08  305.50000  306.89000  301.30000  304.31000   4725510
       2017-11-09  302.50000  304.46000  296.30000  302.99000   5440335
       2017-11-10  302.50000  308.36000  301.85000  302.99000   4621912

[26691 rows x 5 columns]

### Basic data information and analysis

In [77]:
df_multi.shape

(26691, 5)

In [78]:
df_multi.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 26691 entries, ('AAPL', Timestamp('1984-09-07 00:00:00')) to ('TSLA', Timestamp('2017-11-10 00:00:00'))
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    26691 non-null  float64
 1   High    26691 non-null  float64
 2   Low     26691 non-null  float64
 3   Close   26691 non-null  float64
 4   Volume  26691 non-null  int64  
dtypes: float64(4), int64(1)
memory usage: 1.4+ MB


In [79]:
df_multi.describe()

,Open,High,Low,Close,Volume
count,26691.000000,26691.000000,26691.000000,26691.000000,2.669100e+04
mean,106.916292,108.054272,105.682690,106.901946,5.999620e+07
std,187.310525,188.767736,185.632213,187.267739,7.616379e+07
min,0.067200,0.067200,0.067200,0.067200,0.000000e+00
25%,2.054500,2.088200,2.021000,2.054500,6.906406e+06
50%,25.090000,25.385000,24.797000,25.099000,3.978104e+07
75%,112.190000,113.735000,111.205000,112.785000,8.320867e+07
max,1126.100000,1135.540000,1124.060000,1132.880000,2.069770e+09


### Checking presence of null values

In [80]:
df_multi.isna().sum()

Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64

### Ensuring all stocks are in the same date range, based on TSLA's data

In [81]:
end_date = pd.Timestamp('2017-11-10')
start_date = pd.Timestamp('2010-06-28')

df = df_multi[(df_multi.index.get_level_values('Date') >= start_date) &
                      (df_multi.index.get_level_values('Date') <= end_date)]
df.loc['AAPL']

,Open,High,Low,Close,Volume
Date,,,,,
2010-06-28,34.191,34.545,33.874,34.360,163083315
2010-06-29,33.856,33.856,32.565,32.806,315472195
2010-06-30,32.877,33.036,32.020,32.212,205878957
2010-07-01,32.542,32.629,31.147,31.823,285057573
2010-07-02,32.103,32.132,31.144,31.623,193495389
...,...,...,...,...,...
2017-11-06,171.750,174.360,171.100,173.630,34901241
2017-11-07,173.290,174.510,173.290,174.180,24424877
2017-11-08,174.030,175.610,173.710,175.610,24451166


### Adding Daily Return, 7 day Moving Average, 30 day Moving Average and 30 day Rolling Volatility columns

In [82]:
grouped = df.groupby(level='Ticker')

df['Target'] = grouped['Close'].transform(lambda x: (x.pct_change()).shift(-1))
df['Close_lag_1'] = grouped['Close'].transform(lambda x: x.shift(1))
df['Close_lag_2'] = grouped['Close'].transform(lambda x: x.shift(2))
df['Close_lag_3'] = grouped['Close'].transform(lambda x: x.shift(3))

df['Daily Return'] = grouped['Close'].transform(lambda x: x.pct_change()*100)
df['Return_lag_1'] = grouped['Daily Return'].transform(lambda x: x.shift(1))
df['Return_lag_2'] = grouped['Daily Return'].transform(lambda x: x.shift(2))

df['MA_7'] = grouped['Close'].transform(lambda x: x.rolling(window=7).mean())
df['MA_30'] = grouped['Close'].transform(lambda x: x.rolling(window=30).mean())
df['STD_7'] = grouped['Close'].transform(lambda x: x.rolling(window=7).std())
df['STD_30'] = grouped['Close'].transform(lambda x: x.rolling(window=30).std())

df['Volume_lag_1'] = grouped['Volume'].transform(lambda x: x.shift(1))
df['Volume_avg_7'] = grouped['Volume'].transform(lambda x: x.rolling(7).mean())
df['Volume_change'] = grouped['Volume'].transform(lambda x: x.pct_change())

df['30D RV'] = grouped['Daily Return'].transform(lambda x: x.rolling(30).std())

df['MA_ratio'] = df['MA_7'] / df['MA_30']
df['HL_ratio'] = df['High'] / df['Low']
df['CO_ratio'] = df['Close'] / df['Open']

df.loc['TSLA']

C:\Users\Akshat\AppData\Local\Temp\ipykernel_17288\3067806975.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Target'] = grouped['Close'].transform(lambda x: (x.pct_change()).shift(-1))
C:\Users\Akshat\AppData\Local\Temp\ipykernel_17288\3067806975.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Close_lag_1'] = grouped['Close'].transform(lambda x: x.shift(1))
C:\Users\Akshat\AppData\Local\Temp\ipykernel_17288\3067806975.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

,Open,High,Low,Close,Volume,Target,Close_lag_1,Close_lag_2,Close_lag_3,Daily Return,...,MA_30,STD_7,STD_30,Volume_lag_1,Volume_avg_7,Volume_change,30D RV,MA_ratio,HL_ratio,CO_ratio
Date,,,,,,,,,,,,,,,,,,,,,
2010-06-28,17.00,17.00,17.00,17.00,0,0.405294,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000
2010-06-29,19.00,25.00,17.54,23.89,18783276,-0.002512,17.00,NaN,NaN,40.529412,...,NaN,NaN,NaN,0.0,NaN,inf,NaN,NaN,1.425314,1.257368
2010-06-30,25.79,30.42,23.30,23.83,17194394,-0.078473,23.89,17.00,NaN,-0.251151,...,NaN,NaN,NaN,18783276.0,NaN,-0.084590,NaN,NaN,1.305579,0.924002
2010-07-01,25.00,25.92,20.27,21.96,8229863,-0.125683,23.83,23.89,17.00,-7.847251,...,NaN,NaN,NaN,17194394.0,NaN,-0.521364,NaN,NaN,1.278737,0.878400
2010-07-02,23.00,23.10,18.71,19.20,5141807,-0.160937,21.96,23.83,23.89,-12.568306,...,NaN,NaN,NaN,8229863.0,NaN,-0.375226,NaN,NaN,1.234634,0.834783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-11-06,307.00,307.50,299.01,302.78,6482486,0.010800,306.09,299.26,321.08,-1.081381,...,339.294800,11.869917,17.101748,8893974.0,8.646008e+06,-0.271137,2.216589,0.927003,1.028394,0.986254
2017-11-07,301.02,306.50,300.03,306.05,5286320,-0.005685,302.78,306.09,299.26,1.079992,...,337.988133,11.871700,18.099490,6482486.0,8.405465e+06,-0.184523,2.231713,0.924322,1.021565,1.016710
2017-11-08,305.50,306.89,301.30,304.31,4725510,-0.004338,306.05,302.78,306.09,-0.568535,...,336.766133,11.668170,19.101075,5286320.0,8.472770e+06,-0.106087,2.226119,0.920987,1.018553,0.996105


### Finding the stock with maximum daily returns

In [83]:
avg_returns = df.groupby(level='Ticker')['Daily Return'].mean()
highest_stock = avg_returns.idxmax()
highest_value = avg_returns.max()

print(avg_returns)
highest_stock, highest_value

Ticker
AAPL     0.099931
AMZN     0.140714
GOOGL    0.091188
MSFT     0.085982
TSLA     0.210083
Name: Daily Return, dtype: float64


('TSLA', np.float64(0.21008256936590616))

### Finding the months with max volatility for each stock

In [84]:
max_vol = df.groupby(level='Ticker')['30D RV'].idxmax() 
max_vol_values = df.loc[max_vol, '30D RV']

for (ticker, date), volatility in max_vol_values.items():
    print(f"Stock: {ticker} | Start Date: {(date - pd.Timedelta(days=29)).date()} | End Date: {date.date()} | Max 30D Volatility: {volatility}")

Stock: AAPL | Start Date: 2013-01-14 | End Date: 2013-02-12 | Max 30D Volatility: 3.1692108621594346
Stock: AMZN | Start Date: 2011-11-01 | End Date: 2011-11-30 | Max 30D Volatility: 3.6481448939357337
Stock: GOOGL | Start Date: 2015-07-28 | End Date: 2015-08-26 | Max 30D Volatility: 3.806679978619726
Stock: MSFT | Start Date: 2013-07-31 | End Date: 2013-08-29 | Max 30D Volatility: 2.806424904937462
Stock: TSLA | Start Date: 2010-07-12 | End Date: 2010-08-10 | Max 30D Volatility: 9.445034250061946


In [85]:
df.to_csv('../week1.csv')